# Notebook to Move final dcm files selected for processing to Project Folder

In [ ]:
import shutil
import pickle
import pandas as pd
from l3finder.ingest import *
import os
import multiprocessing
from multiprocessing import get_context
from multiprocessing import set_start_method
from tqdm.notebook import tqdm
from investigate import *

# Custom functions
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [ ]:
cwd = os.getcwd()
output = '/tf/pickles'
data = '/tf/data'
target = '/tf/output'
dump = 2

In [ ]:
df_final = load_object(os.path.join(output,'df_final_dump_2.pkl'))
subjects = load_object(os.path.join(output,'subjects_final_dump_2.pkl'))

In [ ]:
file_mover = functools.partial(
        move_subject,
        df_final=df_final,
        target = target)

In [ ]:
df_final = df_final[df_final['ID'].isin([s.id_ for s in subjects])]
print('Len of df: ', len(df_final))
print('Len of sub: ', len(subjects))

In [ ]:
if __name__=='__main__':
    # Find series images
    print("Moving valid subjects")
    
    def pool_mover(pool, func, candidates):
        return [a for a in tqdm(pool.imap_unordered(func, candidates),total=len(candidates))]
        
    print('Filtering series using ', multiprocessing.cpu_count(), ' cores:')
        
    with get_context("spawn").Pool(processes=10) as p:
        result_list = pool_mover(p, file_mover,subjects)
        p.close()
        p.join()
        
    df_final['Processed'] = result_list

In [ ]:
display(df_final.head(10))

In [ ]:
# Single core
move_subject(sub,df_final,target)

In [ ]:
sub = [s for s in subjects if s.id_ == 'Z1077868'][0]

In [ ]:
def move_subject(sub,df_final,target):
    try:
        df_row = df_final[df_final['ID']==sub.id_]
        srcpath = str(sub.path)
        display(df_row)
        # Create Patient folder,
        targetpath = os.path.join(target,sub.id_)
        try: 
            os.mkdir(targetpath) 
        except Exception:
            pass
            
        series = list(sub.find_series())
        ax_series = [a for a in series if a.id_ == df_row['Axial'].values[0]]
        ##print('Comes here', df_row['Sagittal'].values[0])
        try:
            shutil.copytree(str(ax_series[0].series_path), os.path.join(targetpath,ax_series[0].id_)) 
        except:
            pass
        
        if df_row['Sagittal'].values[0]:
            sag_series = [a for a in series if a.id_ == df_row['Sagittal'].values[0]]
            try:
                shutil.copytree(str(sag_series[0].series_path), os.path.join(targetpath,sag_series[0].id_)) 
            except:
                pass
        return 1
    except Exception as e:
        print(e)
        return 0